In [1]:
%load_ext watermark
import pandas as pd
# import numpy as np

# Extracting land use values

This is a summary of the methods to extract topographical features from vector and points layers using QGIS. Atribute names are translated to the target languages and Null values are either corrected or eliminated.

QGIS is not the optimal tool for this operation if the intention is to automate the aquisition of the data. However for a limited number of users and relatively few sample locations added each year this method is adequate to the task.

__Note: 1'500 meter buffer (circular)__

The land-use chapter in the federal report generated alot of interest. It inspired an academic article and collaboration with Wagenigen Research and University. The principal was applied using an empirical Bayes method with the Solid-Waste-Team at the EPFL.

1. the area of the buffer is  $\pi * 1500²$ or 7065000 m²

## Land-use, land-cover and streets

__Land-cover:__ Are elements such as buildings, forest, orchard or undefined. There is at least one land-cover element for each survey location. If so then it takes 100% of the available dry land. 

__Land-use:__ Are elements that may or may not be in the buffer. These are items like schools, hospitals, water-treatment plants. Individually they are only a small portion of the available dry land. 

### Extracting land-cover and land-use:

For this method we are using the land-cover layer from swissTLM regio

finished columns = slug, attribute , attribute_type, area, dry,	scale

In QGIS:

1. create a buffer around each survey point
   * make sure that the survey location and feature_type is in the attributes of the new buffer layer
   * the survey locations are loaded as points from .csv file
   * reproject the points layer to the project CRS 

2. use the new buffer layer as an overlay to the land-cover layer
   * use the overlay intersection tool
   * select the fields to keep from the buffer (slug and feature type)
   * select the fields to keep from the land-cover layer
   * run the function
   * this creates a temporary layer called _intersection_

3. get the surface area of all the land-cover and land-use features in each buffer of the temporary layer
   * use the field calculator for the attribute table of the layer
   * in the field calculator, make a new field and enter the formula `\$area`
   * for this example the method is elipsoid _bessel 1841 (epsg 7001)_
   * this is set in the properties of the QGIS project
   * Export the layer as .csv

4. verify the land-use features per location
   * drop duplicate values: use location, feature and area to define duplicates
   * attention! different names for lake and reservoir
     * change Stausee to See

5. make a dry land feature
   * this is the surface area of the buffer that is not covered by water
   * substract the area of See from the area of the buffer
   * identify survey locations that have siginifcant water features but are not listed as lakes
  
6. Scale the land-use attributes of interest to the available dry-land
  
__Example making dry land columns and scaling the land-use__

    
```python
# separate locations that are lakes
# recall that feature type has a designator for lakes
lakes = lg[(lg.feature_ty == 'l') | lg.slug.isin(snl)].copy()

# from this subset of data separate the surface area covered by water
# set the slug to the index and substract the surface area of the water
# from the surface area of the buffer
lake_only = lakes[lakes.feature == "See"]
lo = lake_only[["slug", "area"]].set_index("slug")

# substract the lake value from the area of the buffer
lo["dry"] = 7065000 - lo.area
lodry = lo["dry"]

# merge the original land-use data from lakes with the
# the dry land results
lgi = lakes.merge(lo["dry"], left_on="slug", right_index=True)
# remove the lake feature from the features columns
lgi = lgi[lgi.feature != "See"].copy()

# scale the landuse feature to the available dry land
lgi["scale"] = (lgi.area/lgi.dry).round(3)

# repeat the process for locations that do not have a lake feature
# these locations are accounted for above
eliminate = [*snl, *lo.index]
# recuperate all other locations
rivers_parcs = lg[~lg.slug.isin(eliminate)].copy()
# define the dry land as the area of the buffer
rivers_parcs["dry"] = 7065000
# scale the features with the dry land value
rivers_parcs["scale"] = rivers_parcs.area/rivers_parcs.dry

# combine the two results
land_cover = pd.concat([rivers_parcs, lgi])
```
Once the dry land value is calculated for each buffer of the land-cover buffer use the dry-land value to scale the components of the land-use buffer

### Extracting street lengths

The tlmREgio streets or strasse layer is applicable.

__Note:__ the street lenghts are not scaled

finished columns = slug, attribute, attribute_type, length

__Attention:__

1. The streets layer can have many copies of the same street in a buffer
2. The cumulative sum of a street in a layer can exceed the radius of the buffer (windy roads)
3. Dissolving the polylines of each buffer in the layer is essential
4. Drop duplicate values on slug, attribute, length

1. Using the same 1'500 m buffer that was used for land-cover and land-use. Intersect the buffer with the street layer
2. Dissolve the polylines in each buffer
   * select the fields from the streets layer to keep (OBJVAL)
   * select the fields from the buffer layer to kepp (slug, feature_ty)
   * check the box `keep disjoint features separate`
   * run the function
   * export to .csv

In [2]:
land_cover_data = "data/end_process/land_cover.csv"
land_use_data = "data/end_process/land_use.csv"
street_data = "data/end_process/streets.csv"
intersection_attributes = "data/end_process/river_intersect_lakes.csv"
land_cover = pd.read_csv(land_cover_data)
land_use = pd.read_csv(land_use_data)
streets = pd.read_csv(street_data)
river_intersect_lakes = pd.read_csv(intersection_attributes)

#### Land-cover



Land-cover attributes

In [3]:
land_cover.attribute.unique()

array(['undefined', 'Siedl', 'Wald', 'Reben', 'Obstanlage'], dtype=object)

land-cover attribute translations

```python
land_cover_fr = {
    'undefined': 'Non défini',
    'Siedl': 'Siedl',
    'Wald': 'Forêt',
    'Reben': 'Vignes',
    'Obstanlage': 'Verger'
}

land_cover_en = {
    'undefined': 'Undefined',
    'Siedl': 'Settlement',
    'Wald': 'Forest',
    'Reben': 'Vines',
    'Obstanlage': 'Orchard'
}
```

Land-cover results for one location

In [4]:
land_cover_fr = {
    'undefined': 'Non défini',
    'Siedl': 'Siedl',
    'Wald': 'Forêt',
    'Reben': 'Vignes',
    'Obstanlage': 'Verger'
}

land_cover_en = {
    'undefined': 'Undefined',
    'Siedl': 'Settlement',
    'Wald': 'Forest',
    'Reben': 'Vines',
    'Obstanlage': 'Orchard'
}

In [5]:
land_cover[land_cover.slug == "parc-des-pierrettes"]

,slug,attribute,attribute_type,area,dry,scale
141,parc-des-pierrettes,undefined,land-cover,68285,3832559,0.018
623,parc-des-pierrettes,Siedl,land-cover,3606885,3832559,0.941
624,parc-des-pierrettes,Wald,land-cover,157389,3832559,0.041


#### Land-use

Land-use attributes:

In [6]:
land_use.attribute.unique()

array(['Baumschule', 'Friedhof', 'Schul- und Hochschulareal',
       'Wald nicht bestockt', 'Abwasserreinigungsareal',
       'Historisches Areal', 'Kraftwerkareal', 'Schrebergartenareal',
       'Truppenuebungsplatz', 'Unterwerkareal',
       'Kehrichtverbrennungsareal', 'Spitalareal',
       'Oeffentliches Parkareal', 'Messeareal',
       'Massnahmenvollzugsanstaltsareal', 'Kiesabbauareal',
       'Steinbruchareal', 'Klosterareal', 'Deponieareal', 'Antennenareal',
       'Lehmabbauareal'], dtype=object)

Land-use translations and groups

```python

land_use_fr = {
    'Baumschule': 'Pépinière',
    'Friedhof': 'Cimetière',
    'Schul- und Hochschulareal': 'Zone scolaire et universitaire',
    'Wald nicht bestockt': 'Forêt non peuplée',
    'Abwasserreinigungsareal': 'Zone de traitement des eaux usées',
    'Historisches Areal': 'Zone historique',
    'Kraftwerkareal': 'Zone de centrale électrique',
    'Schrebergartenareal': 'Zone de jardins familiaux',
    'Truppenübungsplatz': 'Terrain d\'entraînement militaire',
    'Unterwerkareal': 'Zone de sous-station',
    'Kehrichtverbrennungsareal': 'Zone d\'incinération des déchets',
    'Spitalareal': 'Zone d\'hôpital',
    'Öffentliches Parkareal': 'Zone de parc public',
    'Messeareal': 'Zone d\'exposition',
    'Massnahmenvollzugsanstaltsareal': 'Zone d\'établissement de traitement',
    'Kiesabbauareal': 'Zone d\'extraction de gravier',
    'Steinbruchareal': 'Zone de carrière',
    'Klosterareal': 'Zone de monastère',
    'Deponieareal': 'Zone de décharge',
    'Antennenareal': 'Zone d\'antennes',
    'Lehmabbauareal': 'Zone d\'extraction d\'argile'
}

land_use_en = {
    'Baumschule': 'Nursery',
    'Friedhof': 'Cemetery',
    'Schul- und Hochschulareal': 'School and University Area',
    'Wald nicht bestockt': 'Non-stocked Forest',
    'Abwasserreinigungsareal': 'Wastewater Treatment Area',
    'Historisches Areal': 'Historical Area',
    'Kraftwerkareal': 'Power Plant Area',
    'Schrebergartenareal': 'Allotment Garden Area',
    'Truppenübungsplatz': 'Military Training Ground',
    'Unterwerkareal': 'Substation Area',
    'Kehrichtverbrennungsareal': 'Waste Incineration Area',
    'Spitalareal': 'Hospital Area',
    'Öffentliches Parkareal': 'Public Park Area',
    'Messeareal': 'Exhibition Area',
    'Massnahmenvollzugsanstaltsareal': 'Correctional Facility Area',
    'Kiesabbauareal': 'Gravel Extraction Area',
    'Steinbruchareal': 'Quarry Area',
    'Klosterareal': 'Monastery Area',
    'Deponieareal': 'Landfill Area',
    'Antennenareal': 'Antenna Area',
    'Lehmabbauareal': 'Clay Extraction Area'
}

# land_uses_grouped:
# outdoor non technical use:
lu_non_tech = ['Friedhof', 'Hitorisches Areal', 'Schrebergartenareal', 'Öffentliches Parkareal', 'Messeareal', 'Klosterareal',  'Wald nicht bestockt']

# technical-extraction-incineration
lu_extraction = ['Kiesabbauareal', 'Steinbruchareal',  'Lehmabbauareal',]

# waste-water-treatment-powere
lu_technical = ['Kehrichtverbrennungsareal', 'Deponieareal', 'Deponieareal', 'Abwasserreinigungsareal','Unterwerkareal', 'Antennenareal']

# services:
lu_serives = ['Massnahmenvollzugsanstaltsareal', 'Schul- und Hochschulareal', 'Spitalareal']
```

Land-use for one location:

In [7]:
land_use_fr = {
    'Baumschule': 'Pépinière',
    'Friedhof': 'Cimetière',
    'Schul- und Hochschulareal': 'Zone scolaire et universitaire',
    'Wald nicht bestockt': 'Forêt non peuplée',
    'Abwasserreinigungsareal': 'Zone de traitement des eaux usées',
    'Historisches Areal': 'Zone historique',
    'Kraftwerkareal': 'Zone de centrale électrique',
    'Schrebergartenareal': 'Zone de jardins familiaux',
    'Truppenübungsplatz': 'Terrain d\'entraînement militaire',
    'Unterwerkareal': 'Zone de sous-station',
    'Kehrichtverbrennungsareal': 'Zone d\'incinération des déchets',
    'Spitalareal': 'Zone d\'hôpital',
    'Öffentliches Parkareal': 'Zone de parc public',
    'Messeareal': 'Zone d\'exposition',
    'Massnahmenvollzugsanstaltsareal': 'Zone d\'établissement de traitement',
    'Kiesabbauareal': 'Zone d\'extraction de gravier',
    'Steinbruchareal': 'Zone de carrière',
    'Klosterareal': 'Zone de monastère',
    'Deponieareal': 'Zone de décharge',
    'Antennenareal': 'Zone d\'antennes',
    'Lehmabbauareal': 'Zone d\'extraction d\'argile'
}

land_use_en = {
    'Baumschule': 'Nursery',
    'Friedhof': 'Cemetery',
    'Schul- und Hochschulareal': 'School and University Area',
    'Wald nicht bestockt': 'Non-stocked Forest',
    'Abwasserreinigungsareal': 'Wastewater Treatment Area',
    'Historisches Areal': 'Historical Area',
    'Kraftwerkareal': 'Power Plant Area',
    'Schrebergartenareal': 'Allotment Garden Area',
    'Truppenübungsplatz': 'Military Training Ground',
    'Unterwerkareal': 'Substation Area',
    'Kehrichtverbrennungsareal': 'Waste Incineration Area',
    'Spitalareal': 'Hospital Area',
    'Öffentliches Parkareal': 'Public Park Area',
    'Messeareal': 'Exhibition Area',
    'Massnahmenvollzugsanstaltsareal': 'Correctional Facility Area',
    'Kiesabbauareal': 'Gravel Extraction Area',
    'Steinbruchareal': 'Quarry Area',
    'Klosterareal': 'Monastery Area',
    'Deponieareal': 'Landfill Area',
    'Antennenareal': 'Antenna Area',
    'Lehmabbauareal': 'Clay Extraction Area'
}

# land_uses_grouped:
# outdoor non technical use:
lu_non_tech = ['Friedhof', 'Hitorisches Areal', 'Schrebergartenareal', 'Öffentliches Parkareal', 'Messeareal', 'Klosterareal',  'Wald nicht bestockt']

# technical-extraction-incineration
lu_extraction = ['Kiesabbauareal', 'Steinbruchareal',  'Lehmabbauareal',]

# waste-water-treatment-powere
lu_technical = ['Kehrichtverbrennungsareal', 'Deponieareal', 'Deponieareal', 'Abwasserreinigungsareal','Unterwerkareal', 'Antennenareal']

# services:
lu_serives = ['Massnahmenvollzugsanstaltsareal', 'Schul- und Hochschulareal', 'Spitalareal']

In [8]:
land_use[land_use.slug == "parc-des-pierrettes"]

,slug,attribute,attribute_type,area,dry,scale
685,parc-des-pierrettes,Abwasserreinigungsareal,land-use,49488,3832559,0.012913
686,parc-des-pierrettes,Friedhof,land-use,11325,3832559,0.002955
687,parc-des-pierrettes,Oeffentliches Parkareal,land-use,175591,3832559,0.045816
688,parc-des-pierrettes,Schrebergartenareal,land-use,17306,3832559,0.004516
689,parc-des-pierrettes,Schul- und Hochschulareal,land-use,1189161,3832559,0.310279


#### Streets

Streets attributes:

In [9]:
streets.attribute.unique()

array(['Autostr', 'Fahrstraes', 'Fussweg', 'HauptStrAB6', 'VerbindStr4',
       'NebenStr3', 'VerbindStr6', 'Autobahn', 'HauptStrAB4', 'NebenStr6',
       'Autob_Ri'], dtype=object)

Streets: translations and groups

```python
streets_fr = {
    'Autostr': 'autoroute',
    'NebenStr3': 'route secondaire 3',
    'NebenStr6': 'route secondaire 6',
    'HauptStrAB6': 'route principale 6',
    'HauptStrAB4': 'route principale 4',
    'Fahrstraes': 'chemin carrossable',
    'Fussweg': 'chemin pédestre',
    'Autobahn': 'autoroute',
    'Autob_Ri': 'autoroute',
    'VerbindStr4': 'route de liason 4',
    'VerbindStr6': 'route de liason 6',   
}

streets_en = {
    'Autostr': 'freeway',
    'NebenStr3': 'surface streets 3',
    'NebenStr6': 'surface streets 3 6',
    'HauptStrAB6': 'inter regional 6',
    'HauptStrAB4': 'inter regional 4',
    'Fahrstraes': 'bridle path',
    'Fussweg': 'pedestrian trail',
    'Autobahn': 'freeway',
    'Autob_Ri': 'freeway',
    'VerbindStr4': 'intra regional 4',
    'VerbindStr6': 'intra regional 6',   
}

```

The streets at one survey location:

In [10]:
streets_fr = {
    'Autostr': 'autoroute',
    'NebenStr3': 'route secondaire 3',
    'NebenStr6': 'route secondaire 6',
    'HauptStrAB6': 'route principale 6',
    'HauptStrAB4': 'route principale 4',
    'Fahrstraes': 'chemin carrossable',
    'Fussweg': 'chemin pédestre',
    'Autobahn': 'autoroute',
    'Autob_Ri': 'autoroute',
    'VerbindStr4': 'route de liason 4',
    'VerbindStr6': 'route de liason 6',   
}

streets_en = {
    'Autostr': 'freeway',
    'NebenStr3': 'surface streets 3',
    'NebenStr6': 'surface streets 3 6',
    'HauptStrAB6': 'inter regional 6',
    'HauptStrAB4': 'inter regional 4',
    'Fahrstraes': 'bridle path',
    'Fussweg': 'pedestrian trail',
    'Autobahn': 'freeway',
    'Autob_Ri': 'freeway',
    'VerbindStr4': 'intra regional 4',
    'VerbindStr6': 'intra regional 6',   
}

str_surface = ['NebenStr3', 'NebenStr6']
str_ped_br = ['Fahrstraes', 'Fussweg']
str_main = [ 'HauptStrAB6', 'VerbindStr4','VerbindStr6', 'HauptStrAB4', 'NebenStr6']
str_auto = ['Autobahn', 'Autostr', 'Autob_Ri']

In [11]:
streets[streets.slug == "parc-des-pierrettes"]

,slug,attribute_type,attribute,length
638,parc-des-pierrettes,streets,Autobahn,1685
639,parc-des-pierrettes,streets,Fahrstraes,1046
640,parc-des-pierrettes,streets,Fussweg,3257
641,parc-des-pierrettes,streets,HauptStrAB6,2918
642,parc-des-pierrettes,streets,NebenStr3,2850
643,parc-des-pierrettes,streets,VerbindStr4,67
644,parc-des-pierrettes,streets,VerbindStr6,2864


## Rivers: intersection, size and class

This requires the 1'500 m buffer created at the begininng and the rivers layer and the lakes layer.

### intersection, size and class
1. Create a points layer of the intersection of rivers to lakes
   * Keep only points that are labeled as river (fluss, fluss_u)
   * Use the attribute table to select the points
2. Overlay the 1 500 m buffer on the new layer and collect all the intersections within each buffer
   * export to .csv
3. Check for intersections that do not have lake name
   * not all lakes and rivers have names in the layer
   * find the correct name for the lake and add it to the record
   * replace river names with nan value with "no name"

### distance to intersection

__Note:__ the intersection points layer needs to be available

1. Using the `hub lines` tool in QGIS draw lines from survey location to intersect location
   * Limit the distance to the radius of the buffer
   * join on the slug field in both layers
   * eliminate duplicates
   * calculate the length of the line with `$length` in field calculator of the attribute table
   * export to .csv
  
2. Identify locations (slugs) that are missing either the river_name or lake
   * use the previous results (intersection, size and class) to fill in the missing values
   * check that slugs not in the distance matrix are truly without intersections 


In [12]:
river_intersect_lakes[river_intersect_lakes.slug == "parc-des-pierrettes"]

,slug,lake,river_name,size,class,distance,attribute_type
72,parc-des-pierrettes,Le Léman,La Mèbre,8,8,333,river intersection


In [13]:
%watermark -a hammerdirt-analyst -co --iversions

Author: hammerdirt-analyst

conda environment: cantonal_report

pandas: 2.0.3

